 Importação de Bibliotecas


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression  # Para validação
from sklearn.metrics import r2_score, mean_squared_error

 Carregamento e Pré-Processamento dos Dados

In [3]:
# Carrega os dados
df = pd.read_csv('ML_Uber_dados_tratados2.csv')  

df = df.dropna(subset=['num_distancia_m_1', 'num_preco'])


X = df[['num_distancia_m_1', 'num_tempo_estimado_seg_1']]
y = df['num_preco']

print(df.columns)

# Verifica se há dados suficientes
if len(X) < 2:
    raise ValueError("Dados insuficientes para regressão após limpeza.")

Index(['cod_produto', 'cod_provedor', 'cod_categoria', 'dsc_produto',
       'cod_corrida', 'dat_criacao', 'cod_companhia', 'num_preco',
       'dat_atualizacao', 'dsc_carro', 'num_latitude_origem',
       'num_longitude_origem', 'num_latitude_destino', 'num_longitude_destino',
       'num_distancia_m_1', 'num_tempo_estimado_seg_1'],
      dtype='object')


Implementação Manual do Método dos Mínimos Quadrados

In [4]:
# Adiciona coluna de 1s para o intercepto (formato matricial)
X_matrix = np.c_[np.ones((len(X), 1)), X]  

# Calcula coeficientes usando a equação normal: θ = (XᵀX)⁻¹Xᵀy
try:
    theta = np.linalg.inv(X_matrix.T @ X_matrix) @ X_matrix.T @ y  
    print(f"Coeficientes calculados (intercepto, inclinação): {theta.flatten()}")
except np.linalg.LinAlgError:
    # Fallback para SVD se a matriz for singular
    theta = np.linalg.pinv(X_matrix.T @ X_matrix) @ X_matrix.T @ y  
    print(f"Coeficientes via pseudoinversa (SVD): {theta.flatten()}")

Coeficientes calculados (intercepto, inclinação): [4.04740054e+00 1.73841945e-03 9.77577408e-04]


Função de Previsão

In [5]:
def predict_price(distance_m):
    """Prediz o preço com base no modelo treinado."""
    return theta[0] + theta[1] * distance_m 

#usuario coloca a distância em metros
print("Previsão de preço com base na distância em metros (M)")
dist = float(input("Digite a distância em metros: "))
# Exemplo de uso:
print(f"Preço previsto para {dist} M: R${predict_price(dist):.2f}")

Previsão de preço com base na distância em metros (M)
Preço previsto para 5000.0 M: R$12.74


Validação com scikit-learn

In [ ]:
# Comparação com sklearn para verificar correção
model = LinearRegression().fit(X, y)
print(f"Validação sklearn - Intercepto: {model.intercept_}, Coeficiente: {model.coef_[0]}")




Validação sklearn - Intercepto: 4.047400537918232, Coeficiente: 0.0017384194458374862


Avaliação do Modelo

In [8]:
# Calcula os valores previstos
y_pred = X_matrix @ theta

# Cálculo de métricas
r2 = r2_score(y, y_pred)
mse = mean_squared_error(y, y_pred)

print(f"Coeficiente de Determinação (R²): {r2:.3f}")
print(f"Erro Quadrático Médio (MSE): {mse:.3f}")
print(f"Equação do modelo: Preço = {theta[0]:.2f} + {theta[1]:.2f} * Distância")

Coeficiente de Determinação (R²): 0.752
Erro Quadrático Médio (MSE): 243.642
Equação do modelo: Preço = 4.05 + 0.00 * Distância
